# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.53'

In [33]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\Spectra'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [34]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [35]:
df2.head()

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,10mol_FIE_R3,102.792317,1285.966474,980.191989,1285.966474,2766.400685,1.078726,19.783235,0.594958,...,1.420940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1mol_FIA_R1,102.754819,1286.319385,2544.295738,1286.319385,5415.867034,0.854121,25.954015,0.453335,...,0.506374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1mol_FIA_R2,102.709397,1286.479920,2159.117789,1286.479920,4961.339851,0.976415,17.922014,0.303296,...,1.599663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1mol_FIA_R3,102.742824,1286.541341,1954.226293,1286.541341,4806.826294,1.011823,25.919609,0.392802,...,0.514601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1mol_FIB_R1,102.740070,1285.149558,2030.472934,1285.149558,5132.075610,1.057016,16.174576,0.342398,...,0.889430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [36]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window', 'CRR'],
 sort=True)
diad_meta

['10mol_FID_R1.txt',
 '10mol_FID_R1_CRR_DiadFit.txt',
 '10mol_FID_R2.txt',
 '10mol_FID_R2_CRR_DiadFit.txt',
 '10mol_FID_R3.txt',
 '10mol_FID_R3_CRR_DiadFit.txt',
 '10mol_FIE_R1.txt',
 '10mol_FIE_R1_CRR_DiadFit.txt',
 '10mol_FIE_R2.txt',
 '10mol_FIE_R2_CRR_DiadFit.txt',
 '10mol_FIE_R3.txt',
 '10mol_FIE_R3_CRR_DiadFit.txt',
 '1mol_FIA_R1.txt',
 '1mol_FIA_R1_CRR_DiadFit.txt',
 '1mol_FIA_R2.txt',
 '1mol_FIA_R2_CRR_DiadFit.txt',
 '1mol_FIA_R3.txt',
 '1mol_FIA_R3_CRR_DiadFit.txt',
 '1mol_FIB_R1.txt',
 '1mol_FIB_R1_CRR_DiadFit.txt',
 '1mol_FIB_R2.txt',
 '1mol_FIB_R2_CRR_DiadFit.txt',
 '1mol_FIB_R3.txt',
 '1mol_FIB_R3_CRR_DiadFit.txt',
 '4mol_FID_R1.txt',
 '4mol_FID_R1_CRR_DiadFit.txt',
 '4mol_FID_R2.txt',
 '4mol_FID_R2_CRR_DiadFit.txt',
 '4mol_FID_R3.txt',
 '4mol_FID_R3_CRR_DiadFit.txt',
 'AutoCalibration - Laser 532.05 nm - Grating 1800 - Coeff 0.00429475  PASS-postcrash.txt',
 'LL8_404a_r1.txt',
 'LL8_404a_r1_CRR_DiadFit.txt',
 'LL8_404a_r2.txt',
 'LL8_404a_r2_CRR_DiadFit.txt',
 'LL8_404a_r

## Now get all the important information out of the metadata files

In [13]:
meta=pf.loop_convert_datastamp_to_metadata(path=spectra_path, 
files=diad_meta, creation=False,
modification=True)
meta

,filename,date,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,10mol_FID_R1_CRR_DiadFit.txt,"March 23, 2023",March,23,NaN,NaN,NaN,NaN,NaN,10:53:28,39208,NaN
0,10mol_FID_R2_CRR_DiadFit.txt,"March 23, 2023",March,23,NaN,NaN,NaN,NaN,NaN,10:53:29,39209,NaN
0,10mol_FID_R3_CRR_DiadFit.txt,"March 23, 2023",March,23,NaN,NaN,NaN,NaN,NaN,10:53:29,39209,NaN
0,10mol_FIE_R1_CRR_DiadFit.txt,"March 23, 2023",March,23,NaN,NaN,NaN,NaN,NaN,10:53:29,39209,NaN
0,10mol_FIE_R2_CRR_DiadFit.txt,"March 23, 2023",March,23,NaN,NaN,NaN,NaN,NaN,10:53:29,39209,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
0,LL8_631_b_r3.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,8:10:20,29420,NaN
0,LL8_631_b_r2.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,8:3:40,29020,NaN
0,LL8_624_r1.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,8:48:35,31715,NaN
0,LL8_624_r2.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,8:54:37,32077,NaN


## Then get a simple file name you can stitch with the spectra

In [14]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

'10mol_FID_R1_CRR_DiadFit'

## Now get filename from spectra into same form as metadata

In [15]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

In [16]:
file_s

array(['10mol_FIE_R3', '1mol_FIA_R1', '1mol_FIA_R2', '1mol_FIA_R3',
       '1mol_FIB_R1', '1mol_FIB_R2', '1mol_FIB_R3', '4mol_FID_R1',
       '4mol_FID_R2', '4mol_FID_R3', 'LL8_404a_r1', 'LL8_404a_r2',
       'LL8_404a_r3', 'LL8_612_a_r1', 'LL8_612_a_r2', 'LL8_612_a_r3',
       'LL8_612_c_r1', 'LL8_612_c_r2', 'LL8_612_c_r3', 'LL8_615_r1',
       'LL8_615_r2', 'LL8_615_r3', 'LL8_622B_R1', 'LL8_622B_R2',
       'LL8_622B_R3', 'LL8_624_r1', 'LL8_624_r2', 'LL8_624_r3',
       'LL8_626_MI_r1', 'LL8_626_MI_r2', 'LL8_626_MI_r3', 'LL8_631_a_r1',
       'LL8_631_a_r2', 'LL8_631_a_r3', 'LL8_631_b_r1', 'LL8_631_b_r2',
       'LL8_631_b_r3'], dtype='<U13')

In [17]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [19]:
df_combo.head()

,Unnamed: 0,filename_x,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,0,10mol_FIE_R3,102.792317,1285.966474,980.191989,1285.966474,2766.400685,1.078726,19.783235,0.594958,...,January,1,NaN,NaN,NaN,NaN,NaN,1:53:30,6810,NaN
1,0,1mol_FIA_R1,102.754819,1286.319385,2544.295738,1286.319385,5415.867034,0.854121,25.954015,0.453335,...,January,1,NaN,NaN,NaN,NaN,NaN,3:15:6,11706,NaN
2,0,1mol_FIA_R2,102.709397,1286.479920,2159.117789,1286.479920,4961.339851,0.976415,17.922014,0.303296,...,January,1,NaN,NaN,NaN,NaN,NaN,3:22:7,12127,NaN
3,0,1mol_FIA_R3,102.742824,1286.541341,1954.226293,1286.541341,4806.826294,1.011823,25.919609,0.392802,...,January,1,NaN,NaN,NaN,NaN,NaN,3:28:32,12512,NaN
4,0,1mol_FIB_R1,102.740070,1285.149558,2030.472934,1285.149558,5132.075610,1.057016,16.174576,0.342398,...,January,1,NaN,NaN,NaN,NaN,NaN,2:31:39,9099,NaN


In [20]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [21]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [22]:
df_combo.to_excel('FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [24]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None


In [27]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')



In [29]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo

### Add some more useful columns

In [30]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))

In [31]:
df_combo_sec_phase.to_excel('FI_fitting_w_sec_phases.xlsx')